In [1]:
# from pycardano import PaymentSigningKey

# payment_signing_key = PaymentSigningKey.generate()
# payment_signing_key.save("payment.skey")


In [2]:
from pycardano import PaymentSigningKey, PaymentVerificationKey, Address, Network, BlockFrostChainContext, Transaction

blockfrost_api_key = "mainnetp2CEmC4HLrI6ER01KTLdMdRmqSEAkQIa"

context = BlockFrostChainContext(
    project_id=blockfrost_api_key,
    base_url="https://cardano-mainnet.blockfrost.io/api/",
)


In [3]:

tx_hash = "aca8e877b2c806057ceb74110255f5ba6fa9443cc9d26aba9ba2eb22af16c7b8"
tx = context.api.transaction(tx_hash)

# Fetch UTXOs for transaction
order_utxos = context.api.transaction_utxos(tx_hash)

metadata = context.api.transaction_metadata(tx_hash)


if metadata[0].label != '674':
    print("input is not a swap")

if metadata[0].json_metadata.msg[0] != "Minswap: Market Order":
    print("input is not a swap")



In [ ]:
from blockfrost.utils import Namespace
def sum_utxos_amount(utxos: list[Namespace], only:str=None) -> dict:
    total = {}
    for u in utxos:
        if only is None or u.address == only:
            d = total.get(u.address, {})
            for t in u.amount:
                d[t.unit] = float(d.get(t.unit, 0)) + float(t.quantity)
            total[u.address] = d
    return total

def get_change_amount(tx_list: list[str], only:str=None) -> dict:
    utxos_inputs = []
    utxos_outputs = []
    for tx_hash in tx_list:
        utxo = context.api.transaction_utxos(tx_hash)
        utxos_inputs += utxo.inputs
        utxos_outputs += utxo.outputs
    

    total_in = sum_utxos_amount(utxos_inputs, only)
    total_out = sum_utxos_amount(utxos_outputs, only)
    # change = {}
    for addr, amount in total_in.items():
        addr_out = total_out.get(addr, {})
        for t, v in amount.items():
            amount_change = addr_out.get(t, 0) - v
            if amount_change == 0:
                addr_out.pop(t)
            else:
                addr_out[t] = amount_change 
        if len(addr_out.keys()) == 0:
            total_out.pop(addr)
    change = total_out
    return change

# sum_utxos_amount(order_utxos.inputs)

get_change_amount(
    [
        # "aca8e877b2c806057ceb74110255f5ba6fa9443cc9d26aba9ba2eb22af16c7b8",
        "bbeae27f4df816a3f2ffd57e4a0e0ae69e42f424e927a712adf847a84c3a1d3e"
    ]
)
# addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm

{'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': 2000000.0,
  'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': 2903572.0},
 'addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4rvaau66j2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq777e2a': {'lovelace': 5000000.0,
  'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': -2903572.0},
 'addr1q9dhugez3ka82k2kgh7r2lg0j7aztr8uell46kydfwu3vk6n8w2cdu8mn2ha278q6q25a9rc6gmpfeekavuargcd32vsvxhl7e': {'lovelace': -4964266.0}}

In [ ]:
get_change_amount(
    [
        "4c6ff99c832851602489d0aa8e946082746b10ef8ec7f3546d97b22c8d053659",
        # "bbeae27f4df816a3f2ffd57e4a0e0ae69e42f424e927a712adf847a84c3a1d3e"
    ]
)


{'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': 2000000.0,
  '29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e': 221921537.0},
 'addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4rvaau66j2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq777e2a': {'29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e': -221921537.0,
  'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': 2000000.0},
 'addr1q9dhugez3ka82k2kgh7r2lg0j7aztr8uell46kydfwu3vk6n8w2cdu8mn2ha278q6q25a9rc6gmpfeekavuargcd32vsvxhl7e': {'lovelace': -4973525.0}}

In [ ]:
context.api.add("addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4rvaau66j2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq777e2a")

ApiError: {'status_code': 400, 'error': 'Bad Request', 'message': 'Invalid or malformed stake address format.'}

In [ ]:
order_in = {}

for i in order_utxos.inputs:
    d = order_in.get(i.address, {})
    for t in i.amount:
        d[t.unit] = float(d.get(t.unit, 0)) + float(t.quantity)
    order_in[i.address] = d
order_out = {}
for o in order_utxos.outputs:
    d = order_out.get(o.address, {})
    for t in o.amount:
        d[t.unit] = float(d.get(t.unit, 0)) + float(t.quantity)
    order_out[o.address] = d

# user
user = list(order_in.keys())[0]

amount_in = {}
for t, v in order_in[user].items():
    a = float(v) - float(order_out.get(user, {}).get(t, 0))
    if a > 0:
        amount_in[t] = a
    
order_in, order_out, amount_in
# {'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': '4200000'}}

({'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': 11620182.0}},
 {'addr1z8p79rpkcdz8x9d6tft0x0dx5mwuzac2sa4gm8cvkw5hcnzj2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pqsfywsm': {'lovelace': 7700000.0},
  'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': 3738137.0}},
 {'lovelace': 7882045.0})

In [ ]:
# utxos = context.utxos("addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm")
# for utxo in utxos:
#     print("UTxO:", utxo)
#     break


In [45]:
tx_hash = "bbeae27f4df816a3f2ffd57e4a0e0ae69e42f424e927a712adf847a84c3a1d3e"
tx = context.api.transaction(tx_hash)
# Fetch UTXOs for transaction
excute_utxos = context.api.transaction_utxos(tx_hash)

metadata = context.api.transaction_metadata(tx_hash)

if metadata[0].label != '674':
    print("input is not a swap")

if metadata[0].json_metadata.msg[0] != "Minswap: Order Executed":
    print("input is not a swap")


In [46]:
execute_in = {}

for i in excute_utxos.inputs:
    d = execute_in.get(i.address, {})
    for t in i.amount:
        d[t.unit] = float(d.get(t.unit, 0)) + float(t.quantity)
    execute_in[i.address] = d
execute_out = {}
for o in excute_utxos.outputs:
    d = execute_out.get(o.address, {})
    for t in o.amount:
        d[t.unit] = float(d.get(t.unit, 0)) + float(t.quantity)
    execute_out[o.address] = d


amount_out = {}
for t, v in execute_out[user].items():
    a = float(v) - float(execute_in.get(user, {}).get(t, 0))
    if a > 0:
        amount_out[t] = a
    
# execute_in, execute_out, 
amount_out
# {'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': '4200000'}}

{'lovelace': 2000000.0,
 'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': 2903572.0}

In [48]:
execute_in, execute_out

({'addr1w86cprpvnyxcdkj5hlyhmzwwumh6yrxcgctpvdv50rvkknqyhz3rc': {'lovelace': 1961050.0,
   'f5808c2c990d86da54bfc97d89cee6efa20cd8461616359478d96b4c4d534753': 1.0},
  'addr1qyqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqzj2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq6a0h00': {'lovelace': 99030870.0},
  'addr1q9dhugez3ka82k2kgh7r2lg0j7aztr8uell46kydfwu3vk6n8w2cdu8mn2ha278q6q25a9rc6gmpfeekavuargcd32vsvxhl7e': {'lovelace': 284575192.0},
  'addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4rvaau66j2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq777e2a': {'lovelace': 1767044256721.0,
   'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': 1033895878788.0,
   'f5808c2c990d86da54bfc97d89cee6efa20cd8461616359478d96b4c4d5350': 1.0,
   'f5808c2c990d86da54bfc97d89cee6efa20cd8461616359478d96b4c7dd6988c5a86693c76aeec1ea94afa41770be0de21a775ca7a2a1eabdb6a0171': 9.223370905072595e+18},
  'addr1z8p79rpkcdz8x9d6tft0x0dx5mwuzac2sa4gm8cvkw5hcnzj2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pqsfyw

In [17]:
execute_out

{'addr1vyrq3xwa5gs593ftfpy2lzjjwzksdt0fkjjwge4ww6p53dqy4w5wm': {'lovelace': '2000000',
  'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': '2903572'},
 'addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4rvaau66j2c79gy9l76sdg0xwhd7r0c0kna0tycz4y5s6mlenh8pq777e2a': {'lovelace': '1767049256721',
  'c48cbb3d5e57ed56e276bc45f99ab39abe94e6cd7ac39fb402da47ad0014df105553444d': '1033892975216',
  'f5808c2c990d86da54bfc97d89cee6efa20cd8461616359478d96b4c4d5350': '1',
  'f5808c2c990d86da54bfc97d89cee6efa20cd8461616359478d96b4c7dd6988c5a86693c76aeec1ea94afa41770be0de21a775ca7a2a1eabdb6a0171': '9223370905072594749'},
 'addr1q9dhugez3ka82k2kgh7r2lg0j7aztr8uell46kydfwu3vk6n8w2cdu8mn2ha278q6q25a9rc6gmpfeekavuargcd32vsvxhl7e': {'lovelace': '137287596'}}

In [ ]:
# utxos.outputs[0]
# utxos.outputs[2]
# utxos.outputs[3]



In [32]:
# utxos.outputs[1]


In [33]:
# utxos.outputs[2]


In [34]:
# utxos.outputs[3]